In [1]:
import matplotlib as plt
import numpy as np
import pandas as pd
import json
import subprocess
from glob2 import glob

In [1]:
import mir_eval

ModuleNotFoundError: No module named 'mir_eval'

In [2]:
dcml_chordtype_map = {
    "M": "major",
    "m": "minor",
    "Mm7": "dominant-7th",
    "o": "diminished",
    "o7": "full-diminished",
    "mm7": "minor-7th",
    "%7": "half-diminished",
    "MM7": "major-7th",
    "+": "augmented",
    "mM7": "minor-major-7th",
    "+7": "augmented-7th",
}

----

In [16]:
slices = glob("inputs/slices/*.csv")


In [22]:
slices[0][14:-4]

'n15op132_03'

In [26]:
def runExp():
    slices = glob("inputs/slices/*.csv")
    pieceNames = [x[14:-4] for x in slices] # THis is cursed, please remove it at once >:(
    for piece in pieceNames:
        jsonPath = "outputs/"+piece+".json"
        cmd = ["stack","run","fullParse","--","-n","1",piece, "All"] 
        print("Running command: " + (" ".join(cmd)))
        print("Expecting results in " + jsonPath)
        #res = subprocess.run(cmd, cwd="..")
        res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")

        if res.returncode != 0:
            print("Error in subprocess")
            print(res.stderr)
            print(res.stdout)

In [ ]:
#runExp()

## Loading the Dataset
Loads all json files from the given folder

In [7]:
def load_dataset(path):
    jsonFiles = glob(path + '/*.json') #Can be used absolute or relative paths 
    df = pd.DataFrame() 
    for jsonFile in jsonFiles:
        # df = pd.read_json(jsonFile)
        try:
            with open(jsonFile, 'r') as f:
                data = json.loads(f.read())
        except:
            print("Error with {}".format(jsonFile))
            continue

        newdf = pd.json_normalize(data, record_path = ['results'], 
            meta = ['piece'])
        df = pd.concat([df, newdf])
    return df


In [30]:
#x = load_dataset("outputs")

In [31]:
#list(x['piece'].unique())

In [32]:
x.groupby(["piece", "algorithm"]).agg([np.mean, np.std])

/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_31976/1263448163.py:1: FutureWarning: ['chordLabels', 'slices'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x.groupby(["piece", "algorithm"]).agg([np.mean, np.std])


accuracy        likelihood    
                                  mean std          mean std
piece        algorithm                                      
n01op18-1_01 RandomParseSBS   0.565432 NaN -3.209892e+06 NaN
             RandomSample     0.027160 NaN -3.456812e+06 NaN
             RandomSampleSBS  0.360494 NaN -1.728411e+06 NaN
n01op18-1_03 RandomParseSBS   0.630542 NaN -4.926271e+05 NaN
             RandomSample     0.024631 NaN -1.970465e+06 NaN
...                                ...  ..           ...  ..
n16op135_04  RandomSample     0.016667 NaN -5.952403e+06 NaN
             RandomSampleSBS  0.311905 NaN -1.428588e+06 NaN
shortest     RandomParseSBS   0.750000 NaN -1.407877e+01 NaN
             RandomSample     0.000000 NaN -2.194791e+01 NaN
             RandomSampleSBS  0.750000 NaN -1.326117e+01 NaN

[144 rows x 4 columns]

In [18]:
x.groupby(["algorithm"]).agg([np.mean, np.std])

/var/folders/zq/32m4qp9s339211sg4wxlr7dr0000gn/T/ipykernel_31976/607374105.py:1: FutureWarning: ['chordLabels', 'slices', 'piece'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x.groupby(["algorithm"]).agg([np.mean, np.std])


accuracy              likelihood              
                     mean       std          mean           std
algorithm                                                      
RandomParseSBS   0.513342  0.127909 -3.682068e+06  6.731450e+06
RandomSample     0.019084  0.011501 -1.231712e+07  1.356395e+07
RandomSampleSBS  0.316787  0.108288 -2.834742e+06  6.242748e+06